In [67]:
# !pip install --upgrade google-cloud-bigquery-storage

In [68]:
from google.colab import auth
auth.authenticate_user()
from google.cloud import storage, bigquery

project_id = 'wpac-449904'
# Create a storage client
client = bigquery.Client(project=project_id)

In [69]:
query = """
SELECT * FROM `wpac-449904.wpac_data.programs`
"""
programs = client.query(query).to_dataframe()

# Display first few rows
programs.head()


,data_careerId,name,institution,hours,createdAt,updatedAt,costZone,cost,onetCode,institutionType,careerName,programType,providerName,location_coordinates_latitude,location_coordinates_longitude,careerFields,industries,catalog,format
0,628aed0212545d84f96e737a,System Administration and IT Infrastructure Se...,6287b2bbc916a6d2f86f0ae5,Flexible,2022-05-23 06:10:10+00:00,2024-01-25 07:07:59+00:00,"$2,500 - $4,999",3465,15-1244.00,MOOC,Network and Computer Systems Administrators,Short-term training 0-6 months,Coursera,NA,NA,Technology,Information Technology,Remote,Online
1,628aee0412545d84f96e8bcf,CompTIA A+,6287b402192823522fef76d7,Full-time,2022-05-23 06:14:28+00:00,2024-01-25 07:02:12+00:00,"$2,500 - $4,999",3999,15-1232.00,MOOC,Computer User Support Specialists,Short-term training 0-6 months,CompTIA,NA,NA,Technology,Information Technology,Remote,Online
2,62cce3937495cda19099ff9b,DevOps Engineer,6287b5c92b58adbc7b4de7ac,Part-time,2022-07-12 06:59:31+00:00,2024-01-25 06:48:31+00:00,"$1 - $2,499",420,15-1253.00,NA,Software Quality Assurance Analysts and Testers,Short-term training 7+ months,SkillUp - Microsoft Learn,NA,NA,Technology,Business Management & Administration,Remote,Online
3,62cce3937495cda19099ff9b,DevOps Engineer,6287b5c92b58adbc7b4de7ac,Part-time,2022-07-12 06:59:31+00:00,2024-01-25 06:48:31+00:00,"$1 - $2,499",420,15-1253.00,NA,Software Quality Assurance Analysts and Testers,Short-term training 7+ months,SkillUp - Microsoft Learn,NA,NA,Technology,Information Technology,Remote,Online
4,628aee4312545d84f96e918e,Medical Billing and Coding (Voucher Included),6287b2c6c916a6d2f86f0b64,Flexible,2022-05-23 06:15:31+00:00,2024-01-25 07:00:51+00:00,"$2,500 - $4,999",2995,29-2072.00,Community College,Medical Records Specialists,Short-term training 7+ months,Delgado Community College,29.9860719,-90.1035581,Healthcare,Healthcare,Louisiana,Online


In [70]:
programs.drop(columns=['createdAt', 'updatedAt', 'cost', 'onetCode', 'providerName', 'location_coordinates_latitude', 'location_coordinates_longitude'], axis=1, inplace=True)

programs.head()

,data_careerId,name,institution,hours,costZone,institutionType,careerName,programType,careerFields,industries,catalog,format
0,628aed0212545d84f96e737a,System Administration and IT Infrastructure Se...,6287b2bbc916a6d2f86f0ae5,Flexible,"$2,500 - $4,999",MOOC,Network and Computer Systems Administrators,Short-term training 0-6 months,Technology,Information Technology,Remote,Online
1,628aee0412545d84f96e8bcf,CompTIA A+,6287b402192823522fef76d7,Full-time,"$2,500 - $4,999",MOOC,Computer User Support Specialists,Short-term training 0-6 months,Technology,Information Technology,Remote,Online
2,62cce3937495cda19099ff9b,DevOps Engineer,6287b5c92b58adbc7b4de7ac,Part-time,"$1 - $2,499",NA,Software Quality Assurance Analysts and Testers,Short-term training 7+ months,Technology,Business Management & Administration,Remote,Online
3,62cce3937495cda19099ff9b,DevOps Engineer,6287b5c92b58adbc7b4de7ac,Part-time,"$1 - $2,499",NA,Software Quality Assurance Analysts and Testers,Short-term training 7+ months,Technology,Information Technology,Remote,Online
4,628aee4312545d84f96e918e,Medical Billing and Coding (Voucher Included),6287b2c6c916a6d2f86f0b64,Flexible,"$2,500 - $4,999",Community College,Medical Records Specialists,Short-term training 7+ months,Healthcare,Healthcare,Louisiana,Online


In [71]:
programs['programType'] = programs['programType'].replace({'Short-term training 0-6 months':'1', 'Short-term training 7+ months':'2', 'Apprenticeship':'3', 'NA':'0'})

print(programs['programType'].value_counts(dropna=False))

programType
1    2414
2    1730
3     383
0       4
Name: count, dtype: int64


In [72]:
cost_map = {
    '$1 - $2,499':'1',
    '$2,500 - $4,999':'2',
    'Free':'0',
    '$5,000 - $7,499':'3',
    '$7,500 - $9,999':'4',
    '$10,000 +':'5',
    'NA':'0'
}
programs['costZone'] = programs['costZone'].replace(cost_map)

print(programs['costZone'].value_counts(dropna=False))

costZone
1    2174
2    1114
0     616
3     417
4     143
5      67
Name: count, dtype: int64


In [73]:
programs['hours'] = programs['hours'].replace({'Flexible':'1', 'Part-time':'0', 'Full-time':'2'})
programs['hours'].value_counts(dropna=False)

,count
hours,
2,2950
0,905
1,676


In [74]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [75]:
for col in programs.columns:
  programs[col] = programs[col].astype('category')

programs.dtypes


,0
data_careerId,category
name,category
institution,category
hours,category
costZone,category
institutionType,category
careerName,category
programType,category
careerFields,category
industries,category


In [76]:
programs.to_csv('cleaned_programs.csv', index=False)

from google.colab import files
# files.download('cleaned_programs.csv')

In [77]:
dataset_id = "wpac_data"

client = bigquery.Client(project=project_id)
datasets = [dataset.dataset_id for dataset in client.list_datasets()]
print("Available Datasets:", datasets)

if dataset_id not in datasets:
    print(f"Dataset {dataset_id} does not exist. Creating it now...")
    dataset_ref = client.dataset(dataset_id)
    dataset = bigquery.Dataset(dataset_ref)
    dataset.location = "US"  # Change if necessary
    client.create_dataset(dataset, exists_ok=True)
    print(f"Dataset {dataset_id} created.")


Available Datasets: ['wpac_data']


In [78]:
import pandas_gbq

dataset_table = "wpac_data.cleaned_programs"  # Replace with dataset.table name

# Load your CSV into a DataFrame
csv_file_path = "cleaned_programs.csv"  # Replace with your actual file path
# df = pd.read_csv(csv_file_path)

# Use `pandas_gbq.to_gbq` to upload DataFrame
pandas_gbq.to_gbq(
    programs,
    destination_table=dataset_table,
    project_id=project_id,
    if_exists="replace"  # "fail", "replace", "append"
)


100%|██████████| 1/1 [00:00<00:00, 6853.44it/s]
